In [52]:
import numpy as np
import camb
from matplotlib import pyplot as plt
import time
import autograd as ad

In [53]:
#cosmic model 
def get_spectrum(pars,data_lenght,lmax=3000):
    H0=pars[0]
    ombh2=pars[1]
    omch2=pars[2]
    tau=pars[3]
    As=pars[4]
    ns=pars[5]
    pars=camb.CAMBparams()
    pars.set_cosmology(H0=H0,ombh2=ombh2,omch2=omch2,mnu=0.06,omk=0,tau=tau)
    pars.InitPower.set_params(As=As,ns=ns,r=0)
    pars.set_for_lmax(lmax,lens_potential_accuracy=0)
    results=camb.get_results(pars)
    powers=results.get_cmb_power_spectra(pars,CMB_unit='muK')
    cmb=powers['total']
    tt=cmb[:,0]    #you could return the full power spectrum here if you wanted to do say EE
    return tt[2:][:data_lenght]

In [54]:
#We use this to compute the Jacobian 
#It uses the central difference method
def central_diff(fun,pars,dp,x):

    data_lenght = len(x)

    #Initialzaing the A
    A=np.empty([data_lenght,len(pars)])

    #We want the partial with respect to each parameter
    #So we loop over each parameter
    for i in range(len(pars)):
        p_copy=pars.copy() #copying the paramater array 

        #Foward step f(x+dx)
        p_copy[i]=pars[i]+dp[i]
        foward_step=fun(p_copy,data_lenght)

        #Backward step f(x-dx)
        p_copy[i]=pars[i]-dp[i]
        backward_step=fun(p_copy, data_lenght)

        #Central difference
        A[:,i]=(foward_step-backward_step)/(2*dp[i])
    return A

In [55]:
#This perfroms Gauss-Newton method
def newton(fun,pars,N_inv,dp,x,y,niter=20):

    data_lenght = len(x)

    for i in range(niter):

        #Coputing residuals
        pred=fun(pars,data_lenght)
        r=y-pred

        #Compute Jacobian
        J=central_diff(fun,pars,dp,x)

        #The next step is equal to (J^T*N*J)*(J^T*N*resdiuals)^-1
        #As seen in class
        lhs=J.T@N_inv@J
        rhs=J.T@N_inv@r
        cov = np.linalg.inv(lhs)
        step=cov@rhs

        #Increasing parameters by next step 
        pars=pars+step
        
    return pars, cov

In [56]:
#This return the chi square of our model
#provided the parameters 
def chi_sq(fun,parms,errs,y):
    data_lenght = l(y)
    resid = y-fun(parms, data_lenght)
    return np.sum((resid/errs)**2)

In [57]:
def get_step(trial_step):
        L=np.linalg.cholesky(trial_step)
        return L@np.random.randn(trial_step.shape[0])

In [58]:
def mcmc(fun,running_pars,trial_step,data,nstep=20000,T=1):

    #Initializing the arrays that is going to keep 
    # a history of our chain anc chi-square
    chain_history=np.zeros([nstep,len(running_pars)])
    chain_history[0,:]=running_pars #First element is our guess parameters

    chisq_history=np.zeros(nstep)
    running_chi=fun(running_pars,data)
    chisq_history[0]=running_chi  #First element is the chi-square evaluated 
    #with our guess parameter

    for i in range(1,nstep):

        #Add a step to our parameters using a step maker function
        new_pars=running_pars+get_step(trial_step)

        #Compute chi-square with new parameters 
        new_chi=fun(new_pars,data)

        #Check with accept the step using the formula from class
        accept_prob=np.exp(-0.5*(new_chi-running_chi)/T)
        if np.random.rand(1)<accept_prob:
            running_pars=new_pars
            running_chi=new_chi
        
        #If the the step got accepted the next element in our history
        # is the this new step otherwise it's just the previous one 
        chain_history[i,:]=running_pars
        chisq_history[i]=running_chi


    return chain_history,chisq_history

Question 2

In [59]:
pars=np.asarray([69, 0.022, 0.12,0.06, 2.1e-9, 0.95])
planck=np.loadtxt('COM_PowerSpect_CMB-TT-full_R3.01.txt',skiprows=1)
ell=planck[:,0]
spec=planck[:,1]
errs=0.5*(planck[:,2]+planck[:,3])

#Assuming unccorolated noise our noise matrix is 
#simply a diagonal matrix with error^2 at on the diagonal
#But we want want the inverse so I'll multiply by 1/errs^2
N_inv = np.eye(len(errs))*1/errs**2


pars, cov = newton(get_spectrum,pars,N_inv,pars*1e-8,ell,spec)


In [69]:
outpt= open("planck_fit_params.txt", "w")
outpt.write(str(pars))
outpt.write("\n")
for i in range(6):
    outpt.write(str(cov[i]))
    outpt.write("\n")
outpt.close()